In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows',2500)

In [108]:
data = pd.read_excel("NewDataset.xlsx")

In [109]:
data
data = data.drop("Unnamed: 0",axis=1)

In [110]:
test = data.sort_values('countries')
test = test.drop(['pf_rol_procedural','pf_rol_civil','pf_rol_criminal'],axis=1)
test = test.fillna(method='backfill',axis = 0)

In [111]:
test.sort_values('PIB',inplace=True,ascending=False)

In [112]:
def classifier(PIB):
    PIB_third = len(PIB)/3
    r = []
    for i in range(len(PIB)):
        if(i < PIB_third * 1):
            r.append('0')
        elif(i < PIB_third *2):
            r.append('1')
        else:
            r.append('2')
    return r

# 2 ultimos anos da Venezuela vazios
# Dropando países: Taiwan, Macedonia, Syria, Swaziland

In [113]:
test.dropna(how='any',subset=['PIB'],inplace=True)

Criando as Labels e classificando os países como a 0, 1 e 2

In [174]:
test['Label'] = classifier(test['PIB'].values)
final = test.drop(test.columns[:4],axis = 1)
final.drop("PIB",axis = 1,inplace=True)
final = round(final,2)
final = final.fillna(method='ffill')
#np.where(np.isnan(final))

In [115]:
from sklearn.model_selection import train_test_split

In [122]:
x = final.drop("Label",axis = 1)
y = final["Label"]
x_train,X_val,y_train,Y_val = train_test_split(x,y,random_state = 42)

In [123]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest

In [124]:
RFC = RandomForestClassifier(verbose=1)

In [125]:
RFC.fit(x_train,y_train)

/home/arch-gabe/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [126]:
RFC.score(X_val,Y_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.9123287671232877

In [127]:
from sklearn.model_selection import RandomizedSearchCV

In [166]:
RFC = RandomForestClassifier()
hyper_param = {
    "n_estimators": [20,40,50],
    "criterion"  :["gini","entropy"],
    "max_depth"  :[3,10,20,None]
}

In [167]:
rsc = RandomizedSearchCV(RFC,hyper_param, verbose=1, random_state=42,n_jobs=-1)

In [168]:
rsc.fit(x_train,y_train)

/home/arch-gabe/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    1.8s finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [20, 40, 50], 'criterion': ['gini', 'entropy'], 'max_depth': [3, 10, 20, None]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [169]:
rsc.best_score_

0.9222323879231473

In [191]:
rsc.best_params_

{'n_estimators': 50, 'max_depth': None, 'criterion': 'gini'}

In [189]:
final_2 = test.sort_values(['year','PIB'],ascending=False)
final_2_2016 = final_2[final_2['year'] == 2016]
final_2_rest = final_2[final_2['year'] != 2016]
final_2_2016.drop(test.columns[:4],axis = 1)
final_2_2016.drop("PIB",axis = 1,inplace=True)
final_2_rest.drop(test.columns[:4],axis = 1)
final_2_rest.drop("PIB",axis = 1,inplace=True)
x_data = final_2_rest.drop("Label",axis = 1)
y_data = final_2_rest["Label"]
x = final_2_2016.drop("Label",axis = 1)
y = final_2_2016["Label"]
x_train,X_val,y_train,Y_val = train_test_split(x_data,y_data,random_state = 42)

/home/arch-gabe/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [190]:
x_data

,year,ISO_code,countries,region,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,pf_ss_disappearances_violent,pf_ss_disappearances_organized,pf_ss_disappearances_fatalities,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
250,2015,LUX,Luxembourg,Western Europe,8.230306,9.712394,10.0,10.000000,7.50,10.000000,...,7.918760,8.357440,9.383487,8.008284,7.897112,7.64,27.0,8.484667,13.0,1.0
302,2015,CHE,Switzerland,Western Europe,8.393829,9.725954,10.0,10.000000,10.00,10.000000,...,8.001825,8.373754,9.293812,8.268635,8.485416,8.37,4.0,8.814458,3.0,1.0
273,2015,NOR,Norway,Western Europe,8.619200,9.815379,10.0,10.000000,10.00,10.000000,...,7.927507,9.116229,9.069626,8.134577,7.789073,7.65,25.0,8.519496,10.0,1.0
283,2015,QAT,Qatar,Middle East & North Africa,6.654301,9.848384,10.0,10.000000,10.00,10.000000,...,7.927232,9.970187,9.540418,8.565773,8.358465,7.43,43.0,6.512465,105.0,3.0
232,2015,IRL,Ireland,Western Europe,8.106678,9.753197,10.0,10.000000,7.50,10.000000,...,7.994766,8.471290,9.080835,8.578504,8.617664,8.25,5.0,8.645525,4.0,1.0
317,2015,USA,United States,North America,7.132166,8.014185,10.0,9.932481,10.00,9.954295,...,5.926877,9.603297,8.038368,7.665443,8.713322,7.93,10.0,8.378785,18.0,1.0
167,2015,AUS,Australia,Oceania,7.893906,9.603354,10.0,10.000000,10.00,9.972048,...,6.965338,9.091938,8.823021,8.128533,8.456478,8.00,8.0,8.607373,5.0,1.0
300,2015,SWZ,Swaziland,Sub-Saharan Africa,4.800365,3.083054,10.0,10.000000,7.50,10.000000,...,4.132620,9.022881,8.632462,6.385445,7.625161,6.44,112.0,6.284965,117.0,3.0
292,2015,SGP,Singapore,South Asia,8.044037,9.898830,10.0,10.000000,10.00,10.000000,...,8.746061,9.988960,9.254580,9.258681,8.821299,8.84,2.0,8.226765,25.0,1.0
203,2015,DNK,Denmark,Western Europe,8.723094,9.606237,10.0,10.000000,10.00,9.882701,...,7.821546,9.872664,8.542788,7.914661,8.352166,7.75,16.0,8.543023,8.0,1.0
